<a href="https://colab.research.google.com/github/sametaydn/shakespeare_gpt2/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!pip install gpt-2-simple

In [3]:
import gpt_2_simple as gpt2_simple
from datetime import datetime

In [4]:
gpt2_simple.download_gpt2(model_name='124M')

Fetching checkpoint: 1.05Mit [00:00, 209Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:02, 510kit/s]
Fetching hparams.json: 1.05Mit [00:00, 126Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:27, 5.67Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 460Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:01, 851kit/s]
Fetching vocab.bpe: 1.05Mit [00:01, 850kit/s]


In [5]:
from google.colab import files

In [6]:
gpt2_simple.mount_gdrive()

Mounted at /content/drive


In [7]:
sess = gpt2_simple.start_tf_sess()

In [8]:
gpt2_simple.finetune(sess, dataset="shakespeare2.txt", steps=1000, model_name='124M',
                     sample_every=200, save_every=1000, print_every=10, restore_from='fresh')

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.48s/it]


dataset has 338028 tokens
Training...
[10 | 30.02] loss=3.59 avg=3.59
[20 | 53.80] loss=3.33 avg=3.46
[30 | 76.78] loss=3.33 avg=3.42
[40 | 99.45] loss=3.39 avg=3.41
[50 | 122.38] loss=3.22 avg=3.37
[60 | 145.49] loss=3.47 avg=3.39
[70 | 168.44] loss=2.98 avg=3.33
[80 | 191.31] loss=3.16 avg=3.31
[90 | 214.21] loss=3.21 avg=3.30
[100 | 237.18] loss=3.15 avg=3.28
[110 | 260.19] loss=3.20 avg=3.27
[120 | 283.20] loss=2.83 avg=3.23
[130 | 306.21] loss=2.90 avg=3.21
[140 | 329.23] loss=2.93 avg=3.19
[150 | 352.26] loss=3.20 avg=3.19
[160 | 375.23] loss=3.14 avg=3.18
[170 | 398.21] loss=3.19 avg=3.18
[180 | 421.20] loss=2.93 avg=3.17
[190 | 444.13] loss=2.94 avg=3.16
[200 | 467.11] loss=2.82 avg=3.14
======== SAMPLE 1 ========
 this!

KING HENRY VI:
Amen, I fear; and what shall he of this?

NORFOLK:
To go with them--
KING HENRY VI:
To make a house of him?

EXTON:
Well, he seems to be much bigger than he looks:
he can sit still with all the servants of his court
and not speak a word to those

In [9]:
gpt2_simple.copy_checkpoint_to_gdrive(run_name='run1')

In [10]:
gpt2_simple.generate(sess, run_name='run1', length=300)

'Had a mad fear'--which I confess I had,--
Whom I have beat to pieces this night,
Accompanied by his wife and children:
The tailless seas, the tempest-like storms,
Strangled from off the face of the earth,
How could this Power that looked on me
Be angry, so he should? This Power that looks on me
Is dull, dull and maddening;
And when I show myself dull, dull and dull
I show myself bright, bright and bright,
I show myself cool and dry:
Self-same anger, so flame be his,
Supposing me mild, mild and gentle,
He doth beat you to your bed,
And wakes you, so you too must beat me.
Why, 'tis not the other way; be not so blest
But keep one step in, and he shall be
Subject to your yoke: your prosperity,
Hath not been so well defended
In France, for so 'tis men's turn
Came to his assistance: but I, that do
In comforting my country's weak, base and discontented,
Only show'd the truth of your words,
He that made you sound, sounder; but I, that do
In comforting my blunt friends, do so but sound
The bas

FAST API

In [ ]:
!pip install transformers
!pip install fastapi

In [14]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
app = FastAPI()

class TextIn(BaseModel):
    text: str

class TextOut(BaseModel):
    text: str

model_path = "/content/models/124M"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

@app.post("/generate_text", response_model=TextOut)
def generate_text(text_in: TextIn):
    prompt = text_in.text
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=1000,
        top_p=0.95,
        top_k=60,
        temperature=0.7,
        num_return_sequences=1,
    )
    generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return {"text": generated_text}
